#### IMDb(Internet Movie Database)を使った感想のPositive,Negative分類

##### 使用するアーキテクチャ
- RNN
- LSTM

In [132]:
import random
import numpy as np
import string
import re
from collections import Counter
from typing import List
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from sklearn.metrics import f1_score
from tqdm import tqdm

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [133]:
class Config():
    batch_size=128
    n_epoch=20
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    emb_dim = 100
    hid_dim = 50

    checkpoint_path='model/IMDB.pt'
config=Config()

print(config.device)

cuda:0


In [134]:
def torch_log(x):
    return torch.log(torch.clamp(x,min=1e-10))

In [135]:
train_iter = datasets.IMDB(split='train')

train_iter, valid_iter = train_iter.random_split(
    weights={"train": 0.8, "valid": 0.2},
    seed=seed,
    total_length=len(list(train_iter)),
)

In [136]:
#tokenizerでインスタンス化
tokenizer = get_tokenizer("basic_english")

#counterでインスタンス化
counter = Counter()

#train_iterのlineをtokenizerを使ってcounterに加える
for label, line in train_iter:
    counter.update(tokenizer(line))

#vocabメソッドでvocabularyをインスタンス化
vocabulary = vocab(
    counter,
    min_freq=25,
    specials=('<unk>', '<PAD>', '<BOS>', '<EOS>')
)
# <unk>をデフォルトに設定することにより，min_freq回以上出てこない単語は<unk>になる
vocabulary.set_default_index(vocabulary['<unk>'])

word_num = len(vocabulary)

print(f"単語種数: {word_num}")
print(*vocabulary.get_itos()[:100], sep=', ')

単語種数: 9937
<unk>, <PAD>, <BOS>, <EOS>, i, am, curious, yellow, is, a, and, pretentious, steaming, pile, ., it, doesn, ', t, matter, what, one, s, political, views, are, because, this, film, can, hardly, be, taken, seriously, on, any, level, as, for, the, claim, that, frontal, male, nudity, an, automatic, ,, isn, true, ve, seen, films, with, granted, they, only, offer, some, fleeting, but, where, ?, nowhere, don, exist, same, goes, those, crappy, cable, shows, swinging, in, not, sight, indie, movies, like, brown, bunny, which, we, re, treated, to, site, of, vincent, johnson, trace, pink, visible, chloe, before, crying, (, or, ), matters


In [137]:
def text_transform(_text,max_length=256):
    text=[vocabulary[token] for token in tokenizer(_text)][:max_length-2]
    text=[vocabulary['<BOS>']]+text+[vocabulary['<EOS>']]

    return text,len(text)

def collate_batch(batch):
    label_list,text_list,len_seq_list=[],[],[]

    for _label,_text in batch:
    # torchtext==0.15.1からはnegativeは1，positiveは2なので，-1して{0, 1}にする
        label_list.append(_label-1)

        processed_test,len_seq=text_transform(_text)
        text_list.append(torch.tensor(processed_test))
        len_seq_list.append(len_seq)

    return torch.tensor(label_list),pad_sequence(text_list, padding_value=1).T, torch.tensor(len_seq_list)



In [138]:
train_dataloader=DataLoader(
    list(train_iter),
    batch_size=config.batch_size,
    shuffle=True,
    collate_fn=collate_batch
)

valid_dataloader=DataLoader(
    list(valid_iter),
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=collate_batch
)

#### LSTMとRNNで同じ訓練ループなので関数化する

In [139]:
class EarlyStopper:
    def __init__(self, verbose=True, path=config.checkpoint_path, patience=1):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.__early_stop = False
        self.val_f1_score = -np.Inf
        self.path = path
        
        
    @property
    def early_stop(self):
        return self.__early_stop

    def update(self, val_f1_score, model):
        if self.best_score is None:
            self.best_score = val_f1_score
            self.save_checkpoint(model, val_f1_score)
        elif val_f1_score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.__early_stop = True
        else:
            self.best_score = val_f1_score
            self.save_checkpoint(model, val_f1_score)
            self.counter = 0
    
    def save_checkpoint(self, model, val_f1_score):
        if self.verbose:
            print(f'Validation f1score increased ({self.val_f1_score:.6f} --> {val_f1_score:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_f1_score = val_f1_score
        
    def load_checkpoint(self, model):
        if self.verbose:
            print(f'Loading model from last checkpoint with validation f1score {self.val_f1_score:.6f}')
        model.load_state_dict(torch.load(self.path))
        return model

early_stopping=EarlyStopper(patience=7,verbose=True)

In [140]:
from collections import OrderedDict
def train(net,optimizer,n_epoch=config.n_epoch):
    for epoch in range(n_epoch):
        losses_train=[]
        losses_valid=[]

        net.train()
        n_train=0

        with tqdm(train_dataloader) as pbar_epoch:
            for label,line,len_seq in pbar_epoch:
                net.zero_grad()

                t=label.to(config.device)
                x=line.to(config.device)
                len_seq.to(config.device)

                h=net(x,torch.max(len_seq),len_seq)
                y=torch.sigmoid(h).squeeze()#テンソル配列からサイズが１の次元を消去
                loss=-torch.mean(t*torch_log(y)+(1-t)*torch_log(1-y))

                loss.backward()  # 誤差の逆伝播

                optimizer.step()  # パラメータの更新

                losses_train.append(loss.tolist())

                n_train += t.size()[0]

                pbar_epoch.set_postfix(OrderedDict(train_loss=losses_train))
            # Valid
            t_valid = []
            y_pred = []
            net.eval()
        with tqdm(valid_dataloader) as pbar_epoch:
            for label, line, len_seq in pbar_epoch:

                t = label.to(config.device) # テンソルをGPUに移動
                x = line.to(config.device)
                len_seq.to(config.device)

                h = net(x, torch.max(len_seq), len_seq)
                y = torch.sigmoid(h).squeeze()

                loss = -torch.mean(t*torch_log(y) + (1 - t)*torch_log(1 - y))

                pred = y.round().squeeze()  # 0.5以上の値を持つ要素を正ラベルと予測する

                t_valid.extend(t.tolist())
                y_pred.extend(pred.tolist())

                losses_valid.append(loss.tolist())
                val_f1_score=f1_score(t_valid, y_pred, average='macro')

            early_stopping.update(val_f1_score,model=net)

            print('EPOCH: {}, Train Loss: {:.3f}, Valid Loss: {:.3f}, Validation F1: {:.3f}'.format(
                epoch+1,
                np.mean(losses_train),
                np.mean(losses_valid),
                f1_score(t_valid, y_pred, average='macro')
            ))
            pbar_epoch.set_postfix(OrderedDict(valid_loss=losses_valid,f1_score=f1_score(t_valid, y_pred, average='macro')))
            if early_stopping.early_stop:
                print('Early Stopping!')
                break

    net=early_stopping.load_checkpoint(net)

#### Embedding層の実装
- 単語を離散的なIDから連続的なベクトルに変換する
- 埋め込み行列も学習に使われるパラメータで単語間の類似度を示す
- 埋め込み行列はランダムに初期化する

以下の仮想的な状況を考える<br>
辞書: {'猫': 0, 'は': 1, '魚': 2, 'が': 3, '好き': 4, 'です': 5}

V (埋め込み行列):<br>
0: [0.1, 0.3, 0.2]  # 「猫」の埋め込みベクトル<br>
1: [0.2, 0.1, 0.3]  # 「は」の埋め込みベクトル<br>
2: [0.3, 0.2, 0.1]  # 「魚」の埋め込みベクトル<br>
3: [0.1, 0.2, 0.3]  # 「が」の埋め込みベクトル<br>
4: [0.2, 0.3, 0.1]  # 「好き」の埋め込みベクトル<br>
5: [0.3, 0.1, 0.2]  # 「です」の埋め込みベクトル<br>

これに対して以下の文があったとすると<br>
x (単語ID):<br>
文1: [0, 1, 2, 3, 4, 5]  # 「猫は魚が好きです」<br>
文2: [2, 1, 0, 3, 4, 5]  # 「魚は猫が好きです」<br>

出力のテンソルは以下のようになる<br>

出力テンソル:<br>
文1: [[0.1, 0.3, 0.2], [0.2, 0.1, 0.3], [0.3, 0.2, 0.1], [0.1, 0.2, 0.3], [0.2, 0.3, 0.1], [0.3, 0.1, 0.2]]  # 「猫は魚が好きです」<br>
文2: [[0.3, 0.2, 0.1], [0.2, 0.1, 0.3], [0.1, 0.3, 0.2], [0.1, 0.2, 0.3], [0.2, 0.3, 0.1], [0.3, 0.1, 0.2]]  # 「魚は猫が好きです」

In [141]:
class Embedding(nn.Module):
    def __init__(self,emb_dim,vocab_size):
        super().__init__()
        self.embedding_matrix=nn.Parameter(torch.rand((vocab_size,emb_dim),
                                                    dtype=torch.float))
#Embeddingの実行
    def forward(self,x):
        return F.embedding(x,self.embedding_matrix)

In [142]:
class RNN(nn.Module):
    def __init__(self, in_dim, hid_dim):
        super().__init__()
        self.hid_dim = hid_dim
        glorot = 6 / (in_dim + hid_dim*2)
        self.W = nn.Parameter(torch.tensor(np.random.uniform(
                        low=-np.sqrt(glorot),
                        high=np.sqrt(glorot),
                        size=(in_dim + hid_dim, hid_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([hid_dim]).astype('float32')))

    def function(self, h, x):
        return torch.tanh(torch.matmul(torch.cat([h, x], dim=1), self.W) + self.b)

    def forward(self, x, len_seq_max=0, init_state=None):
        x = x.transpose(0, 1)  # 系列のバッチ処理のため、次元の順番を「系列、バッチ」の順に入れ替える
        state = init_state

        if init_state is None:  # 初期値を設定しない場合は0で初期化する
            state = torch.zeros((x[0].size()[0], self.hid_dim)).to(x.device)

        size = list(state.unsqueeze(0).size())
        size[0] = 0
        output = torch.empty(size, dtype=torch.float).to(x.device)  # 一旦空テンソルを定義して順次出力を追加する

        if len_seq_max == 0:
            len_seq_max = x.size(0)
        for i in range(len_seq_max):
            state = self.function(state, x[i])
            output = torch.cat([output, state.unsqueeze(0)])  # 出力系列の追加
        return output

In [143]:
class SequenceTaggingNet(nn.Module):
    def __init__(self, word_num, emb_dim, hid_dim):
        super().__init__()
        self.emb = Embedding(emb_dim, word_num)
        self.rnn = RNN(emb_dim, hid_dim)
        self.linear = nn.Linear(hid_dim, 1)

    def forward(self, x, len_seq_max=0, len_seq=None, init_state=None):
        h = self.emb(x)
        h = self.rnn(h, len_seq_max, init_state)
        if len_seq is not None:
            # 系列が終わった時点での出力を取る必要があるので len_seq を元に集約する
            h = h[len_seq - 1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)
        return y
    
# RNNのモジュールを使った場合のモデルを構築する
    
class SeqenceTaggingNet2(nn.Module):
    def __init__(self,word_num,emb_dim,hid_dim):
        self.emb=Embedding(emb_dim,word_num)
        self.rnn=nn.RNN(emb_dim,hid_dim,1,batch_first=True)
        self.linear=nn.Linear(hid_dim,1)

    def forward(self, x, len_seq_max=0, len_seq=None, init_state=None):
        h = self.emb(x)
        if len_seq_max > 0:
            h, _ = self.rnn(h[:, 0:len_seq_max, :], init_state)
        else:
            h, _ = self.rnn(h, init_state)
        h = h.transpose(0, 1)
        if len_seq is not None:
            # 系列が終わった時点での出力を取る必要があるので len_seq を元に集約する
            h = h[len_seq - 1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)

        return y

In [144]:
net = SequenceTaggingNet(word_num, config.emb_dim, config.hid_dim)
net.to(config.device)

optimizer = optim.Adam(net.parameters())

train(net, optimizer)

  1%|▏         | 2/157 [00:00<00:16,  9.67it/s, train_loss=[0.7773711681365967, 0.7266910076141357]]

100%|██████████| 157/157 [00:18<00:00,  8.69it/s, train_loss=[0.7773711681365967, 0.7266910076141357, 0.6985199451446533, 0.7040643692016602, 0.7098147869110107, 0.696237325668335, 0.6894975900650024, 0.6836028695106506, 0.6911406517028809, 0.7015177011489868, 0.7033514976501465, 0.6876899600028992, 0.692599892616272, 0.6902141571044922, 0.6902273893356323, 0.6943600177764893, 0.6947373151779175, 0.6907326579093933, 0.691839873790741, 0.6896409392356873, 0.6962161064147949, 0.6905859708786011, 0.6916601657867432, 0.6941063404083252, 0.6852084398269653, 0.6860990524291992, 0.6978064775466919, 0.677242636680603, 0.6982951164245605, 0.7019615173339844, 0.6973644495010376, 0.6887200474739075, 0.6959929466247559, 0.6985212564468384, 0.6910181045532227, 0.7050608396530151, 0.6922539472579956, 0.6885624527931213, 0.6890206336975098, 0.6941676139831543, 0.6854497194290161, 0.6889085173606873, 0.6893584728240967, 0.6926817297935486, 0.6886997222900391, 0.6929694414138794, 0.6879715919494629, 0.

Validation f1score increased (-inf --> 0.462919).  Saving model ...
EPOCH: 1, Train Loss: 0.692, Valid Loss: 0.687, Validation F1: 0.463


100%|██████████| 157/157 [00:17<00:00,  8.75it/s, train_loss=[0.6932771801948547, 0.6553357839584351, 0.6666536331176758, 0.6777725219726562, 0.6985442042350769, 0.676143229007721, 0.6632679104804993, 0.674257755279541, 0.6850392818450928, 0.6649065017700195, 0.6585918664932251, 0.675749659538269, 0.6676524877548218, 0.6720970869064331, 0.6985001564025879, 0.683413565158844, 0.6842633485794067, 0.6856293678283691, 0.6797950863838196, 0.6966791152954102, 0.6754579544067383, 0.6908379793167114, 0.6784708499908447, 0.6740370392799377, 0.6555649042129517, 0.6669313907623291, 0.6809806227684021, 0.6692136526107788, 0.68584144115448, 0.6635119915008545, 0.6540162563323975, 0.6650574207305908, 0.6587514877319336, 0.6916971206665039, 0.697352945804596, 0.664478063583374, 0.6601776480674744, 0.6608054637908936, 0.6431913375854492, 0.6556723713874817, 0.6751958727836609, 0.6462963819503784, 0.6440573930740356, 0.6756724119186401, 0.652511715888977, 0.6478677988052368, 0.632858157157898, 0.661856

Validation f1score increased (0.462919 --> 0.596929).  Saving model ...
EPOCH: 2, Train Loss: 0.656, Valid Loss: 0.638, Validation F1: 0.597


...

EarlyStopping counter: 5 out of 7
EPOCH: 15, Train Loss: 0.146, Valid Loss: 1.111, Validation F1: 0.640


100%|██████████| 157/157 [00:18<00:00,  8.68it/s, train_loss=[0.1190328299999237, 0.19095031917095184, 0.06827342510223389, 0.08685432374477386, 0.13925287127494812, 0.08377478271722794, 0.07908552885055542, 0.10248889774084091, 0.13851036131381989, 0.0939319059252739, 0.11402024328708649, 0.16211852431297302, 0.19203394651412964, 0.16785213351249695, 0.18779706954956055, 0.14278572797775269, 0.06735105812549591, 0.06565693020820618, 0.22068414092063904, 0.21883746981620789, 0.10935014486312866, 0.0713667944073677, 0.07079748809337616, 0.16292545199394226, 0.18652303516864777, 0.15458446741104126, 0.25169092416763306, 0.13351601362228394, 0.09409520030021667, 0.08960872143507004, 0.14156392216682434, 0.23259907960891724, 0.1847418248653412, 0.1002504974603653, 0.13508783280849457, 0.10265891253948212, 0.12965483963489532, 0.08338938653469086, 0.23771797120571136, 0.11080894619226456, 0.07872457802295685, 0.08660173416137695, 0.06586308032274246, 0.16691647469997406, 0.16323524713516235

EarlyStopping counter: 6 out of 7
EPOCH: 16, Train Loss: 0.122, Valid Loss: 1.187, Validation F1: 0.642


100%|██████████| 157/157 [00:18<00:00,  8.72it/s, train_loss=[0.12594513595104218, 0.08535489439964294, 0.16008292138576508, 0.09861099720001221, 0.07156838476657867, 0.08432057499885559, 0.05961477383971214, 0.13416185975074768, 0.20459331572055817, 0.18334564566612244, 0.043547727167606354, 0.07637998461723328, 0.16194242238998413, 0.13841721415519714, 0.18351392447948456, 0.11682888865470886, 0.09305907040834427, 0.11410879343748093, 0.10508385300636292, 0.23490068316459656, 0.17295435070991516, 0.18747764825820923, 0.06838235259056091, 0.07359451055526733, 0.08884681761264801, 0.15003186464309692, 0.062111712992191315, 0.07760965079069138, 0.07516790181398392, 0.06014012172818184, 0.10500684380531311, 0.131808802485466, 0.12141262739896774, 0.09479940682649612, 0.07499152421951294, 0.089419424533844, 0.04571559652686119, 0.05667939782142639, 0.07369452714920044, 0.11084583401679993, 0.17621377110481262, 0.0401175394654274, 0.08132348954677582, 0.1238490641117096, 0.0784286558628082

EarlyStopping counter: 7 out of 7
EPOCH: 17, Train Loss: 0.101, Valid Loss: 1.320, Validation F1: 0.631
Early Stopping!
Loading model from last checkpoint with validation f1score 0.726960


### LSTMを用いた実装

- 入力ゲート: $\hspace{20mm}\boldsymbol{i}_t = \mathrm{\sigma} \left(\boldsymbol{W}_i \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_i\right)$
- 忘却ゲート: $\hspace{20mm}\boldsymbol{f}_t = \mathrm{\sigma} \left(\boldsymbol{W}_f \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_f\right)$  
- 出力ゲート: $\hspace{20mm}\boldsymbol{o}_t = \mathrm{\sigma} \left(\boldsymbol{W}_o \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_o\right)$  
- セル:　　　 $\hspace{20mm}\boldsymbol{c}_t = \boldsymbol{f}_t \odot \boldsymbol{c}_{t-1} + \boldsymbol{i}_t \odot \tanh \left(\boldsymbol{W}_c \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_c\right)$
- 隠れ状態: 　$\hspace{20mm}\boldsymbol{h}_t = \boldsymbol{o}_t \odot \tanh \left(\boldsymbol{c}_t \right)$

In [145]:
class SequenceTaggingNet4(nn.Module):
    def __init__(self, word_num, emb_dim, hid_dim):
        super().__init__()
        self.emb = nn.Embedding(word_num, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, 1, batch_first=True)  # nn.LSTMの使用
        self.linear = nn.Linear(hid_dim, 1)
    
    def forward(self, x, len_seq_max=0, len_seq=None, init_state=None):
        h = self.emb(x)
        if len_seq_max > 0:
            h, _ = self.lstm(h[:, 0:len_seq_max, :], init_state)
        else:
            h, _ = self.lstm(h, init_state)
        h = h.transpose(0, 1)
        if len_seq is not None:
            h = h[len_seq - 1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)
        
        return y

In [146]:

early_stopping=EarlyStopper(patience=7,verbose=True,path='./model/IMBD_LSTM.pt')
net = SequenceTaggingNet4(word_num, config.emb_dim, config.hid_dim)
net.to(config.device)
optimizer = optim.Adam(net.parameters())

train(net, optimizer)

100%|██████████| 157/157 [00:05<00:00, 29.46it/s, train_loss=[0.698729395866394, 0.69236159324646, 0.6948516964912415, 0.6941598653793335, 0.6915837526321411, 0.6916572451591492, 0.695105791091919, 0.6923444271087646, 0.693397581577301, 0.6929824352264404, 0.6925323605537415, 0.695821225643158, 0.6913526058197021, 0.6946244835853577, 0.6820111870765686, 0.6821329593658447, 0.6947636604309082, 0.6897933483123779, 0.6928807497024536, 0.7040039896965027, 0.6910910606384277, 0.7030431628227234, 0.6881298422813416, 0.6914405822753906, 0.6949384212493896, 0.685042142868042, 0.6884819269180298, 0.6966086030006409, 0.6988370418548584, 0.686540961265564, 0.6900820136070251, 0.6863418817520142, 0.6941786408424377, 0.6828898191452026, 0.6957176923751831, 0.6930797696113586, 0.6960475444793701, 0.6846505999565125, 0.7019962072372437, 0.6980888843536377, 0.6930607557296753, 0.6872420310974121, 0.6806379556655884, 0.6852893829345703, 0.6875762939453125, 0.6945504546165466, 0.6893776059150696, 0.6940

Validation f1score increased (-inf --> 0.618530).  Saving model ...
EPOCH: 1, Train Loss: 0.674, Valid Loss: 0.635, Validation F1: 0.619


100%|██████████| 157/157 [00:05<00:00, 29.28it/s, train_loss=[0.6231216192245483, 0.531691312789917, 0.6251899600028992, 0.5958290100097656, 0.49257728457450867, 0.6170104146003723, 0.5878537893295288, 0.6336519122123718, 0.6445177793502808, 0.6853920817375183, 0.6634631752967834, 0.6703441739082336, 0.6430200338363647, 0.6143584251403809, 0.6247613430023193, 0.6119644641876221, 0.6167840957641602, 0.6253660321235657, 0.6049641966819763, 0.5657706260681152, 0.5639371871948242, 0.6088065505027771, 0.6339655518531799, 0.6392848491668701, 0.6250555515289307, 0.642001748085022, 0.5642144083976746, 0.5647358894348145, 0.5980889797210693, 0.575366735458374, 0.561823844909668, 0.6403914093971252, 0.5697743892669678, 0.6176519393920898, 0.5504074096679688, 0.713493824005127, 0.6137531995773315, 0.6155554056167603, 0.5681366324424744, 0.6174235939979553, 0.5614286661148071, 0.5475415587425232, 0.5962712168693542, 0.5776998996734619, 0.5962190628051758, 0.5572700500488281, 0.5584982633590698, 0.

Validation f1score increased (0.618530 --> 0.710577).  Saving model ...
EPOCH: 2, Train Loss: 0.599, Valid Loss: 0.585, Validation F1: 0.711


100%|██████████| 157/157 [00:05<00:00, 29.49it/s, train_loss=[0.5158020257949829, 0.5605247616767883, 0.6384954452514648, 0.5680780410766602, 0.533652663230896, 0.5277923941612244, 0.5930191874504089, 0.5198675394058228, 0.5698422789573669, 0.4568105936050415, 0.5726643800735474, 0.5480008125305176, 0.5547996759414673, 0.551186203956604, 0.6438713669776917, 0.5180065631866455, 0.6609342098236084, 0.6225119233131409, 0.5051686763763428, 0.6304517984390259, 0.5378227829933167, 0.5102876424789429, 0.49635419249534607, 0.5896368026733398, 0.5949442982673645, 0.6382412314414978, 0.6074955463409424, 0.5873839259147644, 0.6252151727676392, 0.5053228139877319, 0.5580157041549683, 0.4482951760292053, 0.5719900131225586, 0.5900362730026245, 0.5571698546409607, 0.5793827772140503, 0.6133642196655273, 0.570102334022522, 0.5539973974227905, 0.5430811643600464, 0.4532851576805115, 0.48410049080848694, 0.482596755027771, 0.5175492763519287, 0.5347731113433838, 0.5517439842224121, 0.5045017004013062, 

...

EarlyStopping counter: 5 out of 7
EPOCH: 17, Train Loss: 0.404, Valid Loss: 0.432, Validation F1: 0.808


100%|██████████| 157/157 [00:05<00:00, 28.78it/s, train_loss=[0.3550812900066376, 0.3061400055885315, 0.38735657930374146, 0.2626553177833557, 0.3475789427757263, 0.30413490533828735, 0.3067575693130493, 0.3578227162361145, 0.3208737075328827, 0.2794742286205292, 0.32502084970474243, 0.33017581701278687, 0.3744429349899292, 0.3325144648551941, 0.2722526788711548, 0.31418871879577637, 0.3491418957710266, 0.3367350101470947, 0.2478785365819931, 0.3996313512325287, 0.20005351305007935, 0.358268678188324, 0.4734545350074768, 0.3035343289375305, 0.24411579966545105, 0.33553236722946167, 0.2906672954559326, 0.2593798339366913, 0.3312363624572754, 0.32127898931503296, 0.39075174927711487, 0.4531291425228119, 0.3855685591697693, 0.30332493782043457, 0.2419230341911316, 0.4161282479763031, 0.34321117401123047, 0.38762661814689636, 0.47415482997894287, 0.3485196530818939, 0.34354305267333984, 0.28322553634643555, 0.30616986751556396, 0.4947284758090973, 0.3482034206390381, 0.30515140295028687, 0

EarlyStopping counter: 6 out of 7
EPOCH: 18, Train Loss: 0.301, Valid Loss: 0.402, Validation F1: 0.839


100%|██████████| 157/157 [00:05<00:00, 28.85it/s, train_loss=[0.2239537537097931, 0.2757214605808258, 0.31779396533966064, 0.2685692310333252, 0.27614787220954895, 0.19396042823791504, 0.2075130045413971, 0.21510444581508636, 0.22341278195381165, 0.3184662461280823, 0.39359867572784424, 0.2981640100479126, 0.21447807550430298, 0.22589808702468872, 0.20788021385669708, 0.22795692086219788, 0.2910388708114624, 0.37914127111434937, 0.3167056739330292, 0.24496227502822876, 0.29274192452430725, 0.3557252287864685, 0.3640207052230835, 0.24155524373054504, 0.23567074537277222, 0.22090908885002136, 0.2670629620552063, 0.2655462622642517, 0.3632611334323883, 0.34909558296203613, 0.31158989667892456, 0.30317625403404236, 0.28374117612838745, 0.23384001851081848, 0.28796178102493286, 0.25833049416542053, 0.2472342848777771, 0.19470423460006714, 0.3033245801925659, 0.2665923237800598, 0.21212485432624817, 0.26442334055900574, 0.28014248609542847, 0.29124388098716736, 0.37434816360473633, 0.2512272

EarlyStopping counter: 7 out of 7
EPOCH: 19, Train Loss: 0.249, Valid Loss: 0.430, Validation F1: 0.834
Early Stopping!
Loading model from last checkpoint with validation f1score 0.851991
